# Shape Measurment: Moments

In this section we will take a look at moment based method for measuring galaxy shapes.

[Interact](/voila/render/moments.ipynb)

In [1]:
import numpy as np
from plot import *
from data import * 

## Data

Let's begin by looking at some dummy images with properties we can easily discern by eye.

In [2]:
@interact(data_name=data_list.keys())
def show_data(data_name):
    
    print(data_list[data_name])

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

## Image Flux

Now we can look at the flux per pixel, $I(x, y)$ , and then dermine the total flux of the image, $F$.

In [3]:
@interact(data_name=data_list.keys())
def show_flux(data_name):
    
    show_image(data_list[data_name], show_flux=True)

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

\begin{equation}
    \large F = \int I(x,y) dx dy
\end{equation}


\begin{equation}
    \large F = \sum_{x,y} I(x,y)
\end{equation}

In [4]:
@interact(data_name=data_list.keys())
def print_total_flux(data_name):
    
    print('F = {}'.format(np.sum(data_list[data_name])))

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

## Centroid

\begin{equation}
    \large \bar{x} = \frac{\sum_{x,y} xI(x,y)}{\sum_{x,y} I(x,y)}
\end{equation}

\begin{equation}
    \large \bar{y} = \frac{\sum_{x,y} yI(x,y)}{\sum_{x,y} I(x,y)}
\end{equation}


In [5]:
def get_centroid(data):
    
    # Sum flux over x and y individually
    sum_i = np.array([np.sum(data, axis=i) for i in (1, 0)])
    
    # Get range of x and y values
    ranges = np.array([np.arange(i) for i in data.shape])
    
    # Calculate centroids
    cents = np.sum(sum_i * ranges, axis=1) / np.sum(data)
    
    return cents.astype(int)

In [6]:
@interact(data_name=data_list.keys())
def show_centroid(data_name):
    
    data = data_list[data_name]
    centroid = get_centroid(data)
    
    show_image(data, centroid=centroid)

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

## Moments

\begin{equation}
    \large Q_{xy} = \frac{\sum_{x,y} I(x,y)(x - \bar{x})(y - \bar{y})}{\sum_{x,y} I(x,y)}
\end{equation}

In [7]:
def get_moments(data):
    
    centroid = get_centroid(data)
    ranges = np.array([np.arange(i) for i in data.shape])
    
    x = np.outer(ranges[0] - centroid[0], np.ones(data.shape[1]))
    y = np.outer(np.ones(data.shape[0]), ranges[1] - centroid[1])
    
    q = np.array([np.sum(data * xi * xj) for xi in (x, y) for xj in (x, y)])
    q = (q / np.sum(data)).reshape(2, 2).astype('complex')
    
    return q

In [8]:
@interact(data_name=data_list.keys())
def print_moments(data_name):
    
    print(get_moments(data_list[data_name]))

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

## Ellipticity

\begin{equation}
    \large \chi = \frac{Q_{00}-Q_{11}+2iQ_{01}}{Q_{00}+Q_{11}}
\end{equation}

\begin{equation}
    \large \epsilon = \frac{Q_{00}-Q_{11}+2iQ_{01}}{Q_{00}+Q_{11}+2\sqrt{Q_{00}Q_{11}-Q_{01}^2}}
\end{equation}

In [9]:
def get_ellipticity(data, method='chi'):
    
    # Calculate moments
    q = get_moments(data)
    
    # Calculate the size.
    r2 = q[0, 0] + q[1, 1]

    # Calculate the numerator
    num = (q[0, 0] - q[1, 1] + 2 * np.complex(0, q[0, 1]))
    
    # Calculate the denominator
    den = r2
    
    if method == 'epsilon':
        den += 2 * np.sqrt(q[0, 0] * q[1, 1] - q[0, 1] ** 2)
    
    # Calculate the ellipticity
    ellip = num / den

    return np.around([ellip.real, ellip.imag], 3)

In [10]:
@interact(data_name=data_list.keys())
def show_centroid(data_name):
    
    data = data_list[data_name]
    centroid = get_centroid(data)
    chi = get_ellipticity(data)
    epsilon = get_ellipticity(data, method='epsilon')
    
    show_image(data, centroid=centroid, chi=chi, epsilon=epsilon)

interactive(children=(Dropdown(description='data_name', options=('Horizontal Line', 'Vertical Line', 'Diagonal…

\begin{equation}
    \large a^2 = \frac{Q_{00}+Q_{11}+\sqrt{(Q_{00}-Q_{11})^2 + 4Q_{01}^2}}{2}
\end{equation}

\begin{equation}
    \large b^2 = \frac{Q_{00}+Q_{11}-\sqrt{(Q_{00}-Q_{11})^2 + 4Q_{01}^2}}{2}
\end{equation}

\begin{equation}
    \large \tan{2\theta} = \frac{2Q_{01}}{Q_{00}-Q_{11}}
\end{equation}

In [11]:
def get_abt(data):
    
    q = get_moments(data)
    
    qq_plus = q[0, 0] + q[1, 1]
    qq_minus = q[0, 0] - q[1, 1]
    root = np.sqrt(qq_minus ** 2 + 4 * q[0, 1] ** 2)
    
    a = np.around(np.real(np.sqrt(0.5 * (qq_plus + root))), 3)
    b = np.around(np.real(np.sqrt(0.5 * (qq_plus - root))), 3)
    if qq_minus == 0.0:
        theta = -45.0 * np.sign(np.real(q[0, 1]))
    else:
        theta = np.around(np.real(0.5 * np.arctan(2 * q[0, 1] / qq_minus)) * 180. / np.pi, 3)
        
    if qq_minus > 0.0 and q[0, 1] == 0.0:
        theta += 90.0
    elif qq_minus > 0.0:
        theta -= 90.0 * np.sign(np.real(q[0, 1]))
    
    return a, b, theta

In [12]:
@interact(a=(1, 5, 1), b=(1, 5, 1), theta=(-90, 90, 15))
def show_ellipse(a, b, theta):
    
    data = make_ellipse(a, b, theta)
    centroid = get_centroid(data)
    chi = get_ellipticity(data)
    epsilon = get_ellipticity(data, method='epsilon')
    abt = get_abt(data)
    
    show_image(data, centroid=centroid, chi=chi, epsilon=epsilon, abt=abt)

interactive(children=(IntSlider(value=3, description='a', max=5, min=1), IntSlider(value=3, description='b', m…